# Titanic: Machine Learning from Disaster
### by Javier Alonso

This notebook faces the project about the Titanic in [Kaggle](https://www.kaggle.com/c/titanic). We have data from all the passengers of the boat and information of some of them about if they died or not. Our goal is to predict if the rest survived or not

The results have been posted in [Medium](https://medium.com/@canceleiro/titanic-machine-learning-from-disaster-2a95fca25390)

## Table of Contents

1. [Preliminary Wrangling](#Preliminary_Wrangling)<br>
1.a.-[Gathering Data](#Gathering_Data)<br>
1.b.-[Assesing and Cleaning Data](#Assesing_cleaning_data)<br>
1.c.-[Consolidation of Dataframes](#Consolidaion_dataframes)<br>
2. [Exploratory Data Analysis](#EDA)<br>
2.a.-[Passengers Characteristics](#Passengers_Characteristics)<br>
2.b.-[Survival Characteristics](#Survival_Characteristics)<br>
3. [Machine Learning Algorithm](#MLA)<br>
3.a.-[Data preparation](#Data_preparation)<br>
3.b.-[Test of different models](#Test_models)<br>
3.c.-[Tune selected model](#Tune_selected_model)<br>
4. [Conclusion](#Conclusion)<br>
5. [Reflection](#Reflection)<br>

## <a class="anchor" id="Preliminary_Wrangling">Part 1: Preliminary Wrangling

In [738]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform as sp_rand
from scipy.stats import uniform as sp_randFloat
from scipy.stats import uniform as sp_randInt
from scipy.stats import uniform as sp_random

### <a class="anchor" id="Gathering_Data"> 1.a.- Gathering Data

In this section we´ll gather data from both files available

In [359]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.shape, test.shape

((1460, 81), (1459, 80))

In [371]:
test.Id.describe()

count    1459.000000
mean     2190.000000
std       421.321334
min      1461.000000
25%      1825.500000
50%      2190.000000
75%      2554.500000
max      2919.000000
Name: Id, dtype: float64

We concatenate both dataframes for having the complete view

In [120]:
df = pd.concat([train, test], sort=False)
df.shape

(2919, 81)

A view of the dataframe

In [12]:
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


The datatypes

### <a class="anchor" id="Assesing_cleaning_data"> 1.b.- Asessing and cleaning data

Now, we´re going to assess the dataframe visually and programmatically for quality and tidiness issues, and do the cleaning of these issues

First thing we do a copy of my dataframe


In [265]:
df_clean = df.copy()

1.- incorrect datatypes: MSSubClass, OverallQual,OverallCond column: it should be a string. We change the datatype

In [266]:
df_clean.MSSubClass = df_clean.MSSubClass.astype(str)
df_clean.OverallQual = df_clean.OverallQual.astype(str)
df_clean.OverallCond = df_clean.OverallCond.astype(str)

#### Fixing Null Values

NaN values in Survived, Age (20%), Fare (1 value), Cabin(77% of values) and Embarked (2 values) column. 

**1.-** The **SalePrice column** doesn´t count as it´s the result variable. We have to see how to fix the other variables

In [209]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null object
MSZoning         2915 non-null object
LotFrontage      2433 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            198 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2917 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null object
OverallCond      2919 non-null object
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2918 n

**2.-** NaN values in **cathegories column** with less than 10% of values being null. We replace the values with the mode. 

In [210]:
round(100*df_clean[['MSZoning','Utilities','Exterior1st','Exterior2nd','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','SaleType','LotFrontage','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','FireplaceQu','GarageYrBlt','GarageCars','GarageArea','PoolQC','Fence','MiscFeature','Alley']].isnull().sum()/len(df_clean),2)

MSZoning         0.14
Utilities        0.07
Exterior1st      0.03
Exterior2nd      0.03
MasVnrType       0.82
BsmtQual         2.77
BsmtCond         2.81
BsmtExposure     2.81
BsmtFinType1     2.71
BsmtFinType2     2.74
Electrical       0.03
KitchenQual      0.03
Functional       0.07
GarageType       5.38
GarageFinish     5.45
GarageQual       5.45
GarageCond       5.45
SaleType         0.03
LotFrontage     16.65
MasVnrArea       0.79
BsmtFinSF1       0.03
BsmtFinSF2       0.03
BsmtUnfSF        0.03
TotalBsmtSF      0.03
BsmtFullBath     0.07
BsmtHalfBath     0.07
FireplaceQu     48.65
GarageYrBlt      5.45
GarageCars       0.03
GarageArea       0.03
PoolQC          99.66
Fence           80.44
MiscFeature     96.40
Alley           93.22
dtype: float64

**cathegories column** with less than 10% of values being null. We replace the values with the mode. 

In [267]:
df_clean['MSZoning'] = df_clean['MSZoning'].fillna(df_clean['MSZoning'].mode().iloc[0])
df_clean['Utilities'] = df_clean['Utilities'].fillna(df_clean['Utilities'].mode().iloc[0])
df_clean['Exterior1st'] = df_clean['Exterior1st'].fillna(df_clean['Exterior1st'].mode().iloc[0])
df_clean['Exterior2nd'] = df_clean['Exterior2nd'].fillna(df_clean['Exterior2nd'].mode().iloc[0])
df_clean['MasVnrType'] = df_clean['MasVnrType'].fillna(df_clean['MasVnrType'].mode().iloc[0])
df_clean['BsmtQual'] = df_clean['BsmtQual'].fillna(df_clean['BsmtQual'].mode().iloc[0])
df_clean['BsmtCond'] = df_clean['BsmtCond'].fillna(df_clean['BsmtCond'].mode().iloc[0])
df_clean['BsmtExposure'] = df_clean['BsmtExposure'].fillna(df_clean['BsmtExposure'].mode().iloc[0])
df_clean['BsmtFinType1'] = df_clean['BsmtFinType1'].fillna(df_clean['BsmtFinType1'].mode().iloc[0])
df_clean['BsmtFinType2'] = df_clean['BsmtFinType2'].fillna(df_clean['BsmtFinType2'].mode().iloc[0])
df_clean['Electrical'] = df_clean['Electrical'].fillna(df_clean['Electrical'].mode().iloc[0])
df_clean['KitchenQual'] = df_clean['KitchenQual'].fillna(df_clean['KitchenQual'].mode().iloc[0])
df_clean['Functional'] = df_clean['Functional'].fillna(df_clean['Functional'].mode().iloc[0])
df_clean['GarageType'] = df_clean['GarageType'].fillna(df_clean['GarageType'].mode().iloc[0])
df_clean['GarageFinish'] = df_clean['GarageFinish'].fillna(df_clean['GarageFinish'].mode().iloc[0])
df_clean['GarageQual'] = df_clean['GarageQual'].fillna(df_clean['GarageQual'].mode().iloc[0])
df_clean['GarageCond'] = df_clean['GarageCond'].fillna(df_clean['GarageCond'].mode().iloc[0])
df_clean['SaleType'] = df_clean['SaleType'].fillna(df_clean['SaleType'].mode().iloc[0])

**cathegories column** with more than 10% of values being null. We delete the columns

In [268]:
df_clean = df_clean.drop(['PoolQC','Fence','MiscFeature','Alley'], axis=1)

**3.-** NaN in **numeric features**. 

In [213]:
round(100*df_clean[['LotFrontage','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','FireplaceQu','GarageYrBlt','GarageCars','GarageArea']].isnull().sum()/len(df_clean),2)

LotFrontage     16.65
MasVnrArea       0.79
BsmtFinSF1       0.03
BsmtFinSF2       0.03
BsmtUnfSF        0.03
TotalBsmtSF      0.03
BsmtFullBath     0.07
BsmtHalfBath     0.07
FireplaceQu     48.65
GarageYrBlt      5.45
GarageCars       0.03
GarageArea       0.03
dtype: float64

**Numeric features** with more than 20% of NaN values. We delete the column

In [269]:
df_clean = df_clean.drop(['FireplaceQu'], axis=1)

**Numeric features** with less than 20% of NaN values. We replace it with the mean

In [270]:
df_clean['LotFrontage'] = df_clean['LotFrontage'].fillna(df_clean['LotFrontage'].mean())
df_clean['MasVnrArea'] = df_clean['MasVnrArea'].fillna(df_clean['MasVnrArea'].mean())
df_clean['BsmtFinSF1'] = df_clean['BsmtFinSF1'].fillna(df_clean['BsmtFinSF1'].mean())
df_clean['BsmtFinSF2'] = df_clean['BsmtFinSF2'].fillna(df_clean['BsmtFinSF2'].mean())
df_clean['BsmtUnfSF'] = df_clean['BsmtUnfSF'].fillna(df_clean['BsmtUnfSF'].mean())
df_clean['BsmtFullBath'] = df_clean['BsmtFullBath'].fillna(df_clean['BsmtFullBath'].mean())
df_clean['BsmtHalfBath'] = df_clean['BsmtHalfBath'].fillna(df_clean['BsmtHalfBath'].mean())
df_clean['TotalBsmtSF'] = df_clean['TotalBsmtSF'].fillna(df_clean['TotalBsmtSF'].mean())
df_clean['GarageYrBlt'] = df_clean['GarageYrBlt'].fillna(df_clean['GarageYrBlt'].mean())
df_clean['GarageCars'] = df_clean['GarageCars'].fillna(df_clean['GarageCars'].mean())
df_clean['GarageArea'] = df_clean['GarageArea'].fillna(df_clean['GarageArea'].mean())

Now we can see there are only NaNs in the variable to predict

In [219]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 76 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null object
OverallCond      2919 non-null object
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 

#### Columns to delete

There are columns that 90% of its values is one of the possible values and they don´t seem features important for the price, so we drop de column these: utilities, street, LandSlope, Condition2, RoofMatl, CentralAir, Electrical, Functional, MiscVal

In [220]:
df_clean.MiscVal.value_counts()

0        2816
400        18
500        13
450         9
600         8
700         7
2000        7
650         3
1200        3
1500        3
4500        2
2500        2
480         2
3000        2
12500       1
300         1
350         1
8300        1
420         1
80          1
54          1
460         1
490         1
3500        1
560         1
17000       1
15500       1
750         1
800         1
900         1
1000        1
1150        1
1300        1
1400        1
1512        1
6500        1
455         1
620         1
Name: MiscVal, dtype: int64

In [221]:
df_clean.Functional.value_counts()

Typ     2719
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: Functional, dtype: int64

In [222]:
df_clean.Electrical.value_counts()

SBrkr    2672
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

In [223]:
df_clean.Electrical.value_counts()

SBrkr    2672
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

In [224]:
df_clean.CentralAir.value_counts()

Y    2723
N     196
Name: CentralAir, dtype: int64

In [225]:
df_clean.RoofMatl.value_counts()

CompShg    2876
Tar&Grv      23
WdShake       9
WdShngl       7
Roll          1
Metal         1
Membran       1
ClyTile       1
Name: RoofMatl, dtype: int64

In [226]:
df_clean.Condition2.value_counts()

Norm      2889
Feedr       13
Artery       5
PosA         4
PosN         4
RRNn         2
RRAn         1
RRAe         1
Name: Condition2, dtype: int64

In [227]:
df_clean.LandSlope.value_counts()

Gtl    2778
Mod     125
Sev      16
Name: LandSlope, dtype: int64

In [228]:
df_clean.Street.value_counts()

Pave    2907
Grvl      12
Name: Street, dtype: int64

In [229]:
df_clean.Utilities.value_counts()

AllPub    2918
NoSeWa       1
Name: Utilities, dtype: int64

In [271]:
df_clean = df_clean.drop(['Utilities', 'Street', 'LandSlope', 'Condition2', 'RoofMatl', 'CentralAir','Electrical', 'Functional', 'MiscVal'], axis=1)

#### Basement Area Simplify

We leave only the total Basement Area and delete the Basement partial areas: BsmtFinSF1, BsmtFinSF2, BsmtUnfSF

In [272]:
df_clean = df_clean.drop(['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'], axis=1)

#### Total Floor Surface

We sum the Surface of First and Second Floor and have the total

In [273]:
df_clean['FlrSf'] = df_clean['1stFlrSF'] + df_clean['2ndFlrSF']
df_clean = df_clean.drop(['1stFlrSF', '2ndFlrSF'], axis=1)

#### Total Number of Bathrooms

We sum the total number of bathroom, total as 1 and half as 0.5

In [274]:
df_clean['Baths'] = df_clean['BsmtFullBath'] + 0.5 *df_clean['BsmtHalfBath'] + df_clean['FullBath'] + 0.5 *df_clean['HalfBath']
df_clean = df_clean.drop(['BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath'], axis=1)

#### Garaje size

We have the garage area and garge number of cars. They seem redundant, so we delete the number of cars

In [277]:
df_clean = df_clean.drop(['GarageCars'], axis=1)

#### Total Porch Area
We sum the total area of porches

In [279]:
df_clean['PorchSF'] = df_clean['OpenPorchSF'] + df_clean['EnclosedPorch'] + df_clean['3SsnPorch'] + df_clean['ScreenPorch']
df_clean = df_clean.drop(['OpenPorchSF', 'EnclosedPorch','3SsnPorch','ScreenPorch'], axis=1)

In [282]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 56 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
LotShape         2919 non-null object
LandContour      2919 non-null object
LotConfig        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null object
OverallCond      2919 non-null object
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-null object
MasVnrType       2919 non-null object
MasVnrArea       2919 non-null float64
ExterQual        2919 non-null object
ExterCond        2919 non-null object
Foundation       2919

### <a class="anchor" id="Consolidaion_dataframes"> 1.c.- Consolidation of dataframes

In this section we´re going to consolidate both dataframes into files, that we´ll read for the next sections

In [283]:
df_clean.to_csv('Prize_info.csv', index = False)

In [287]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 56 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null int64
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
LotShape         2919 non-null object
LandContour      2919 non-null object
LotConfig        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-null object
MasVnrType       2919 non-null object
MasVnrArea       2919 non-null float64
ExterQual        2919 non-null object
ExterCond        2919 non-null object
Foundation       2919 no

## <a class="anchor" id="MLA">Part 3: Machine learning algorithm

### <a class="anchor" id="Data_preparation"> 3.a.- Data preparation
We´re going to take the big dataframe, prepare the dummys.

In [372]:
df = pd.read_csv('Prize_info.csv')

#due to importing issues I´ve to change some datatypes
df.MSSubClass = df.MSSubClass.astype(str)
df.OverallQual = df.OverallQual.astype(str)
df.OverallCond = df.OverallCond.astype(str)



### Algorithm Selection
I´m going to test a few regression algorithms with all the features to see which one works better

In [375]:
#I create the dummies for all cathegorical columns
df1 = pd.get_dummies(df, drop_first = True)

df_train1 = df1[df1.SalePrice.notnull()]
df_test1 = df1[df1.SalePrice.isnull()]

#We create test and train dataframes
X = df_train1.drop("SalePrice", axis=1).copy()
y = df_train1["SalePrice"]
X_K = df_test1.drop("SalePrice", axis=1).copy() #K of Kaggle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [378]:
def test_model(alg,file):
    train_score = 0
    test_score = 0
    clf = alg
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_K = clf.predict(X_K)
    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test) 
    output = pd.DataFrame({'Id': X_K.Id,'SalePrice': y_K})
    output.to_csv(file, index=False)
    
    return train_score, test_score


In [382]:
#Test Linear Regression, create df of scores and its score
model = test_model(LinearRegression(),'Kaggle/00_Linear_Regression.csv')
scores = pd.DataFrame({'Algorithm': 'Linear Regression', 'Train_score': [model[0]], 'Test_score': [model[1]]},columns = ['Algorithm','Train_score','Test_score'])

#Test Lasso, create add score to df
model =test_model(Lasso(),'Kaggle/01_Lasso.csv')
scores = scores.append({'Algorithm' : 'Lasso' , 'Train_score' : model[0], 'Test_score' : model[1]} , ignore_index=True)

#Test Elastic Net, create add score to df
model =test_model(ElasticNet(),'Kaggle/02_Elastic_Net.csv')
scores = scores.append({'Algorithm' : 'Elastic Net' , 'Train_score' : model[0], 'Test_score' : model[1]} , ignore_index=True)

#Test KNeighborsRegressor(), create add score to df
model =test_model(KNeighborsRegressor(),'Kaggle/03_KNeighbors_Regressor.csv')
scores = scores.append({'Algorithm' :'KNeighbors Regressor' , 'Train_score' : model[0], 'Test_score' : model[1]} , ignore_index=True)

#Test DecisionTreeRegressor(), create add score to df
model =test_model(DecisionTreeRegressor(),'Kaggle/04_Decision_Tree_Regressor.csv')
scores = scores.append({'Algorithm' : 'Decision Tree Regressor' , 'Train_score' : model[0], 'Test_score' : model[1]} , ignore_index=True)

#Test GradientBoostingRegressor(), create add score to df
model =test_model(GradientBoostingRegressor(),'Kaggle/05_Gradient_Boosting_Regressor.csv')
scores = scores.append({'Algorithm' : 'Gradient Boosting Regressor' , 'Train_score' : model[0], 'Test_score' : model[1]} , ignore_index=True)


C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 341382695869.4884, tolerance: 696659484.3571945
  positive)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 806037677909.1978, tolerance: 696659484.3571945
  positive)


I´ve uploaded to Kaggle and the results are 

![Scores](Kaggle/Scores_with_no_parameters.jpg)

In [385]:
K_scores = [0.20750, 0.20195, 0.18215, 0.28110, 0.24025,0.15904] 

scores['Kaggle_score'] = K_scores

In [399]:
print("The score of the different algorithms is")
scores.sort_values(by ='Kaggle_score', ascending = True)

The score of the different algorithms is


,Algorithm,Train_score,Test_score,Kaggle_score
5,Gradient Boosting Regressor,0.964071,0.898051,0.15904
2,Elastic Net,0.787717,0.818659,0.18215
1,Lasso,0.899994,0.862483,0.20195
0,Linear Regression,0.900054,0.862063,0.20750
4,Decision Tree Regressor,1.000000,0.658259,0.24025
3,KNeighbors Regressor,0.740940,0.681873,0.28110


The measure of Kaggle and the other scores is different, so don´t have correspondence. In Kaggle, the submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price, while we are using R2 value. But the relative order in scores should be the same.
The results order differ a bit but in both cases the best score is the Gradient Boosting Regressor, so that´s the one we are going to tune in detail.

### Features Selection
I´m going to test the Gradient Boosting Regressor algorithms with some of the the features,not all, to see which one works better. Te previous score 0.987886 was with all the features

**Method 1** Intuition selection of features
I get the correlation to know the best numeric features

In [420]:
numeric_features= df_train.select_dtypes(include=[np.number])
corr=numeric_features.corr()
abs(corr.SalePrice).sort_values(ascending=False)

SalePrice       1.000000
FlrSf           0.716883
GrLivArea       0.708624
Baths           0.631731
GarageArea      0.623431
TotalBsmtSF     0.613581
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
MasVnrArea      0.475210
GarageYrBlt     0.471062
Fireplaces      0.466929
LotFrontage     0.334820
WoodDeckSF      0.324413
LotArea         0.263843
PorchSF         0.195739
BedroomAbvGr    0.168213
KitchenAbvGr    0.135907
PoolArea        0.092404
MoSold          0.046432
YrSold          0.028923
LowQualFinSF    0.025606
Id              0.021917
Name: SalePrice, dtype: float64

In [421]:
# I get the six first numeric features and from the cathegorical I get the Neighborhood
house_features = ['FlrSf','GrLivArea','Baths','GarageArea','TotalBsmtSF','TotRmsAbvGrd',
                  'Neighborhood',      
                 'SalePrice']
df_train1 = df_train[house_features]
df_test1 = df_test[house_features]

#I create the dummies for all cathegorical columns
df_train1 = pd.get_dummies(df_train1, drop_first = True)
df_test1 = pd.get_dummies(df_test1, drop_first = True)

#We create test and train dataframes
X = df_train1.drop("SalePrice", axis=1).copy()
y = df_train1["SalePrice"]
X_K = df_test1.drop("SalePrice", axis=1).copy() #K of Kaggle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#I create the model and score it
clf = GradientBoostingRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_K = clf.predict(X_K)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

scores2 = pd.DataFrame({'Algorithm': 'Gradient Boosting Regressor','Feature Reduction': 'Intuitive Selection',
                        'Train_score': train_score, 'Test_score': test_score},
                       columns = ['Algorithm','Feature Reduction','Train_score','Test_score'], index=[0])

output = pd.DataFrame({'Id': df_test.Id,'SalePrice': y_K})
output.to_csv('Kaggle/06_GBR_feat_intuition.csv', index=False)


In [422]:
print("Test score is:", test_score, "being the original 0.9")

Test score is: 0.876789714971877 being the original 0.9


**Method 2** Using Select Percentile


In [423]:
#I create the dummies for all cathegorical columns
df1 = pd.get_dummies(df, drop_first = True)

df_train1 = df1[df1.SalePrice.notnull()]
df_test1 = df1[df1.SalePrice.isnull()]

#We create test and train dataframes
X = df_train1.drop("SalePrice", axis=1).copy()
y = df_train1["SalePrice"]
X_K = df_test1.drop("SalePrice", axis=1).copy() #K of Kaggle

#I reduce the features
red = SelectPercentile(chi2, percentile=75)
X_new = red.fit_transform(X, y)
X_K_new = red.transform(X_K)

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)


In [424]:
#I create the model and score it
clf = GradientBoostingRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_K = clf.predict(X_K_new)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print("Test score is:", test_score, "being the original 0.9")

scores2 = scores2.append({'Algorithm': 'Gradient Boosting Regressor','Feature Reduction': 'Percentile (70)',
                          'Train_score' :train_score.astype(float), 'Test_score' : test_score.astype(float)}, ignore_index = True)

output = pd.DataFrame({'Id': df_test.Id,'SalePrice': y_K})
output.to_csv('Kaggle/07_GBR_feat_percentiles.csv', index=False)

Test score is: 0.8899013674234095 being the original 0.9


Testing different percentiles, none of them get better results than with all the features

**Method 3** Using PCA


In [698]:
#I create the dummies for all cathegorical columns
df1 = pd.get_dummies(df, drop_first = True)

df_train1 = df1[df1.SalePrice.notnull()]
df_test1 = df1[df1.SalePrice.isnull()]

#We create test and train dataframes
X = df_train1.drop("SalePrice", axis=1).copy()
y = df_train1["SalePrice"]
X_K = df_test1.drop("SalePrice", axis=1).copy() #K of Kaggle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#I test different number of features and keep the one with best restults
pca = PCA(100)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_K_pca = pca.transform(X_K)

In [699]:
#I create the model and score it
clf = GradientBoostingRegressor()
clf.fit(X_train_pca, y_train)
y_pred = clf.predict(X_test_pca)
y_K = clf.predict(X_K_pca)
train_score = clf.score(X_train_pca, y_train)
test_score = clf.score(X_test_pca, y_test)

scores2 = scores2.append({'Algorithm': 'Gradient Boosting Regressor','Feature Reduction': 'PCA (100)',
                          'Train_score' :train_score.astype(float), 'Test_score' : test_score.astype(float)}, ignore_index = True)
print("Test score is:", test_score, "being the original 0.9")

output = pd.DataFrame({'Id': df_test.Id,'SalePrice': y_K})
output.to_csv('Kaggle/08_GBR_feat_PCA.csv', index=False)

Test score is: 0.817264950895713 being the original 0.9


I´ve uploaded to Kaggle and the results are 

![Scores](Kaggle/Scores_with_feature_sleection.jpg)

In [429]:
K_scores = [0.18967, 0.16435, 0.19733] 

scores2['Kaggle_score'] = K_scores

In [430]:
scores2

,Algorithm,Feature Reduction,Train_score,Test_score,Kaggle_score
0,Gradient Boosting Regressor,Intuitive Selection,0.913933,0.876790,0.18967
1,Gradient Boosting Regressor,Percentile (70),0.954502,0.889901,0.16435
2,Gradient Boosting Regressor,PCA (100),0.965081,0.835634,0.19733


The result is worse than having all the features, where we had a test score of 0.9, but in percentile and PCA we will reach, of course, the original result if we had chosen all the features.

**Conclussion** Feature selection is generally used for noise reduction and decrease of computational time. In our case it seems there´s not too much noise, as we can´t improve the original scores, and really speed is not an issue here as computational time is not very high.
In other cases would be interesting to use feature selection just for reducing the time consumption., but not in this one

#### OUTLIER DETECTION
The path we´re going to follow for outlier detection is:
- Train the model with all the train dataset, without splitting into train and test
- Remove points with larger residual error
- Re-Train the model
- Analyse results

For the moment Gradient Boost Regressor with no parameters and all features got the best result, so that´s the one we are going to use

**Train the model**

First we train the model with all the data, without splitting into train and test

In [681]:
def model_no_outliers(perc, file):
    #FIRST WE TRAIN THE MODEL WITH ALL THE DATA, WITHOUT SPLITTING INTO TRAIN AND TEST
    df = pd.read_csv('Prize_info.csv')

    #due to importing issues I´ve to change some datatypes
    df.MSSubClass = df.MSSubClass.astype(str)
    df.OverallQual = df.OverallQual.astype(str)
    df.OverallCond = df.OverallCond.astype(str)

    df1 = df[df.SalePrice.notnull()]

    #Train the model
    #I create the dummies for all cathegorical columns
    df1 = pd.get_dummies(df1, drop_first = True)

    #We create test and train dataframes
    X = df1.drop("SalePrice", axis=1).copy()
    y = df1["SalePrice"]

    # train_score = 0
    # test_score = 0
    clf = GradientBoostingRegressor()
    clf.fit(X, y)
    y_pred = clf.predict(X)
    score = clf.score(X, y)

#     print("Score with outliers",score )

    #WE DELETE THE ROWS
    cleaned_data = pd.DataFrame(columns=['id','prediction','value','error'])
    i = 0
    for n in range(len(y_pred)):
        id = X.Id[i]
        error = y_pred[i] - y.iloc[i]
        error = error*error
        cleaned_data = cleaned_data.append({'id': id,  'prediction' : y_pred[i] , 'value' : y.iloc[i], 'error' : error} , ignore_index=True)
        i = i +1
    # cleaned_data.id = cleaned_data.id.astypÇe(int)

    perc = np.percentile(cleaned_data.error, perc)
    to_drop = cleaned_data[cleaned_data.error > perc].id.tolist()

    df2 = pd.read_csv('Prize_info.csv')
    #due to importing issues I´ve to change some datatypes
    df2.MSSubClass = df2.MSSubClass.astype(str)
    df2.OverallQual = df2.OverallQual.astype(str)
    df2.OverallCond = df2.OverallCond.astype(str)

    #I create the dummies for all cathegorical columns
    df2 = pd.get_dummies(df2, drop_first = True)

    df_train2 = df2[df2.SalePrice.notnull()]
    df_test2 = df2[df2.SalePrice.isnull()]

    indexNames = df2[ df2['Id'].isin(to_drop) ].index
     # Delete these row indexes from dataFrame
    df_train2.drop(indexNames , inplace=True)

    X = df_train2.drop("SalePrice", axis=1).copy()
    y = df_train2["SalePrice"]
    X_K = df_test2.drop("SalePrice", axis=1).copy() #K of Kaggle
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    tr_score =0
    tt_score = 0
    
    clf = GradientBoostingRegressor()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_K = clf.predict(X_K)
    tr_score = round(clf.score(X_train, y_train),2)
    tt_score = round(clf.score(X_test, y_test),2)
        
#     print("Train Score without outliers",tr_score)
#     print("Test Score without outliers",tt_score)
    output = pd.DataFrame({'Id': X_K.Id,'SalePrice': y_K})
    output.to_csv(file, index=False)
    return tr_score, tt_score

In [690]:
#create dataframe of results
scores_outliers = pd.DataFrame(columns=['Algorithm','Outliers_Deleted %','Train_score','Test_score'])

#deleting 10% results
perc = 90
no_outliers = model_no_outliers(90,'Kaggle/09_GBR_NoOutliers_90.csv')
scores_outliers = scores_outliers.append({'Algorithm' : 'GBR' , 'Outliers_Deleted %': (100-perc),'Train_score' :no_outliers[0] , 'Test_score' : no_outliers[1]} , ignore_index=True)

# #deleting 15% results
perc = 85
no_outliers = model_no_outliers(85,'Kaggle/10_GBR_NoOutliers_85.csv')
scores_outliers = scores_outliers.append({'Algorithm' : 'GBR' , 'Outliers_Deleted %': (100-perc),'Train_score' :no_outliers[0] , 'Test_score' : no_outliers[1]} , ignore_index=True)

# #deleting 20% results
perc = 80
no_outliers = model_no_outliers(85,'Kaggle/11_GBR_NoOutliers_80.csv')
scores_outliers = scores_outliers.append({'Algorithm' : 'GBR' , 'Outliers_Deleted %': (100-perc),'Train_score' :no_outliers[0] , 'Test_score' : no_outliers[1]} , ignore_index=True)


C:\Users\Telu Teruno\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


I´ve uploaded to Kaggle and the results are 

![Scores](Kaggle/Scores_with_no_ouliers.jpg)

In [693]:
K_scores = [0.16838, 0.16081, 0.16093] 
scores_outliers['Kaggle_score'] = K_scores

In [694]:
scores_outliers

,Algorithm,Outliers_Deleted %,Train_score,Test_score,Kaggle_score
0,GBR,10,0.98,0.92,0.16838
1,GBR,15,0.98,0.94,0.16081
2,GBR,20,0.98,0.94,0.16093


We´ve been able to get a better result in our test data set removing the outliers. The best 0.15904option would be the second one, removing the points with 15% more erros. On the other hand the Kaggle result hasn´t been improved as the best result has been 0.15904

**Conclussion** Improving our test score possible can´t improve the Kaggle result, but it´s like a double check

### <a class="anchor" id="Tune_selected_model"> 3.c.- **Tune selected model**

For tuning and improving the score of the Gradient Boosting Regression we do a GridSearch with different parameters of the model to improve it

In [709]:
df = pd.read_csv('Prize_info.csv')

#due to importing issues I´ve to change some datatypes
df.MSSubClass = df.MSSubClass.astype(str)
df.OverallQual = df.OverallQual.astype(str)
df.OverallCond = df.OverallCond.astype(str)

In [710]:
#I create the dummies for all cathegorical columns
df1 = pd.get_dummies(df, drop_first = True)

df_train1 = df1[df1.SalePrice.notnull()]
df_test1 = df1[df1.SalePrice.isnull()]

#We create test and train dataframes
X = df_train1.drop("SalePrice", axis=1).copy()
y = df_train1["SalePrice"]
X_K = df_test1.drop("SalePrice", axis=1).copy() #K of Kaggle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [734]:
sp_randint()

In [742]:
model = GradientBoostingRegressor()
parameters = {'learning_rate': sp_randFloat(),
              'subsample'    : sp_randFloat(),
              'n_estimators' : sp_randInt(100, 1000),
              'max_depth'    : sp_randInt(4, 10) 
             }

randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                           cv = 2, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train)

# # Results from Random Search
# print("\n========================================================")
# print(" Results from Random Search " )
# print("========================================================")    

# print("\n The best estimator across ALL searched params:\n",
#       randm.best_estimator_)

# print("\n The best score across ALL searched params:\n",
#       randm.best_score_)

# print("\n The best parameters across ALL searched params:\n",
#       randm.best_params_)
# print("\n ========================================================")
# rsearch_scores = cross_val_score(rsearch,X,y,cv=10)
# score_GBR = rsearch_scores.mean()

# y_pred = rsearch.predict(X_test)
# y_K = rsearch.predict(X_K)
# train_score = rsearch.score(X_train, y_train)
# test_score = rsearch.score(X_test, y_test) 
# # output = pd.DataFrame({'Id': X_K.Id,'SalePrice': y_K})
# # output.to_csv(file, index=False)
# print("Train score:",train_score)
# print("Test score:", test_score)
# print("The best parameters of the tuning are:", cv.best_params_ )

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [703]:
print("The best parameters of the tuning are:", cv.best_params_ )
# print("The score achieved by this model is:",score_logr)

The best parameters of the tuning are: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 120}


In [476]:
gbr = GradientBoostingRegressor()

parameters = {'max_depth': [1, 2,3],
              'n_estimators': [100,120,140],
              'learning_rate': [0.1, 0.5, 1],
             }   

cv = GridSearchCV(gbr, param_grid=parameters)

# train classifier
cv.fit(X, y)

cv_scores = cross_val_score(cv,X,y,cv=10)
score_logr = cv_scores.mean()

Y_K = cv.predict(X_K)


C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: Conve

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

...

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached w

In [481]:
print("The best parameters of the tuning are:", cv.best_params_ )
print("The score achieved by this model is:",score_logr)
#I create the file to send to Kaggle
get_file('Kaggle_results/01_results_LR_tuned.csv')

The best parameters of the tuning are: {'C': 1, 'fit_intercept': True, 'intercept_scaling': 1.5, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
The score achieved by this model is: 0.7957374304846214


The Kaggle score has been the next

![Score](Kaggle_results/End_Score.jpg)


So we have improved the test score from 0.794626 to 0.79573743, but the Kaggle scored hasn´t improved and stayed at 0.74162.

## <a class="anchor" id="Conclusion">Part 4: Conclusion

This has been a very interesting project because we´ve been able to work with real data in a well-known case as it´s the Titanic accident, and we got very clear results in the EDAs and the process of obtaining a Machine Learning Algorithm for predicting the survival or not of the passengers

## <a class="anchor" id="Reflection">Part 5: Reflection

As in the Kaggle results we couldn´t improve our algorithm score (we did it on the test side) a further analysis could be made to investigate detailed tuning on some of the other algorithms that could have a better performance than Logistic Regression, once they are tuned.
